In [3]:
%load_ext zipline

In [1]:
#%%zipline --start 2011-1-1 --end 2013-1-1 -o perf

from zipline.api import order, record, symbol
import matplotlib.pyplot as plt
import zipline as zp
import zipline.utils.factory as zpf
import pandas as pd
import numpy as np
from datetime import datetime

data = zpf.load_from_yahoo(stocks=['AAPL'], 
                           indexes={}, 
                           start=datetime(2014, 1, 1),
                           end=datetime(2014, 2, 1), 
                           adjusted=False)
class BuyApple(zp.TradingAlgorithm):

    def initialize(context):
        pass

    def handle_data(context, data):
        order(symbol('AAPL'), 10)
        record(AAPL=data.current(symbol('AAPL'), 'price'))
        
result = BuyApple().run(data['2014'])
result.tail()

ImportError: No module named 'zipline'

In [253]:
talib.MACDEXT?

In [2]:
aapl_data = zpf.load_from_yahoo(stocks=['AAPL'], 
                           indexes={}, 
                           start=datetime(2016, 6, 25),
                           end=datetime(2016, 9, 10), 
                           adjusted=False)
aapl_data.reset_index(inplace=True)
aapl_data

NameError: name 'zpf' is not defined

In [6]:

import pandas as pd
import talib
import numpy as np

In [7]:
import talib

In [8]:
# here we call on the talib function WMA, which has syntax - wma = talib.WMA(array_of_float_values, window)
# calculate WMA and put it back into a df
#del cl_wma
ma_window = 5
wma = pd.DataFrame(talib.WMA(aapl_data.AAPL.values, ma_window))

NameError: name 'aapl_data' is not defined

In [232]:
# Conctenating two df. 1 - original with dates and close prices and wma. Both should have indexes reset
cl_wma = pd.concat([wma, aapl_data], axis=1)
# .columns here renames the columns
cl_wma.columns = ['WMA', 'Date', 'AAPL_close']

In [248]:
# HMA 
# Step 1 - Calculate WMA for the half of the period and multiply result by 2
HMA_step1 = pd.DataFrame(talib.WMA(aapl_data.AAPL.values, ma_window/2)*2)
HMA_step1

,0
0,NaN
1,186.146662
2,188.260000
3,190.399999
4,191.586664
5,190.579997
6,190.699997
7,191.606669
8,192.866668
9,193.760004


In [251]:
# Step 2 - Calculate WMA for the full period and subtract if the result calculated in the Step 1.
HMA_step2 = pd.DataFrame(talib.WMA(aapl_data.AAPL.values, ma_window) - talib.WMA(aapl_data.AAPL.values, ma_window/2)*2)
talib.WMA(aapl_data.AAPL.values, ma_window)[15:16]

array([ 99.297335])

In [ ]:
# Step 3 - Calculate the square root of the number representing the full period.
HMA_step3 = np.sqrt(ma_window)

In [242]:
# Step 4 - Calculate n days WMA from the result we got in the Step 2 with n being the number we got in the Step 3
HMA_step4 = pd.DataFrame(talib.WMA(talib.WMA(aapl_data.AAPL.values, ma_window) - talib.WMA(aapl_data.AAPL.values, ma_window/2)*2, HMA_step3)).abs()
HMA_step2

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,-96.635332
5,-95.399998
6,-95.307999
7,-95.995336
8,-96.892001
9,-97.394003


In [234]:
# MAMA and FAMA
mama, fama = talib.MAMA(cl_wma.AAPL_close.values, fastlimit=0.5, slowlimit=0.05)
mama = pd.DataFrame({'MAMA' : mama})
fama = pd.DataFrame({'FAMA' : fama})

In [235]:
cl_wma = pd.concat([cl_wma, fama, mama, HMA_step4], axis=1)

In [236]:
cl_wma=cl_wma.rename(columns = {0:'HMA'})

In [237]:
cl_wma

,WMA,Date,AAPL_close,FAMA,MAMA,HMA
0,NaN,2016-06-27 00:00:00+00:00,92.040001,NaN,NaN,NaN
1,NaN,2016-06-28 00:00:00+00:00,93.589996,NaN,NaN,NaN
2,NaN,2016-06-29 00:00:00+00:00,94.400002,NaN,NaN,NaN
3,NaN,2016-06-30 00:00:00+00:00,95.599998,NaN,NaN,NaN
4,94.951332,2016-07-01 00:00:00+00:00,95.889999,NaN,NaN,NaN
5,95.179999,2016-07-05 00:00:00+00:00,94.989998,NaN,NaN,95.811776
6,95.391999,2016-07-06 00:00:00+00:00,95.529999,NaN,NaN,95.338665
7,95.611333,2016-07-07 00:00:00+00:00,95.940002,NaN,NaN,95.766223
8,95.974667,2016-07-08 00:00:00+00:00,96.680000,NaN,NaN,96.593113
9,96.366001,2016-07-11 00:00:00+00:00,96.980003,NaN,NaN,97.226669


In [145]:
%load_ext zipline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


In [151]:
%%zipline --start 2016-1-1 --end 2016-1-10
# This is the call though zipline magic and without extracting data from web as it uses by default the quandl-bundle 
# that we first need to ingest ( download the data into) 

from zipline.api import symbol, order, record

def initialize(context):
    pass

def handle_data(context, data):
    order(symbol(''), 10)
    record(NUGT=data.current(symbol('GLD'), 'price'))

SymbolNotFound: Symbol 'AUX' was not found.